![](./graphs/dl_banner.jpg)

# 用tensorflow low level API构建卷积神经网络
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

## 0.知识背景：卷积神经网络
卷积神经网络是一种在计算机视觉当中广泛应用的神经网络，其特殊的网络结构，包含卷积层、池化层等，能在共享参数的同时保证对图像特征的高效抽取。经典的卷积神经网络结构如下。
![CNN](http://personal.ie.cuhk.edu.hk/~ccloy/project_target_code/images/fig3.png)

## 0.问题背景：MNIST 手写数字
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

这是一个非常经典的问题，我们的对象是手写数字图片，我们需要根据手写数字图片的输入，构建线性的分类器(softmax分类器)去区分图片是手写数字0-9中的哪一个。这个问题的数据集，每一张图片的表示是长宽为28的矩阵，我们有时候会把它展开成784维的向量。MNIST手写数据集长成下面这样。

![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

更多的信息可以参考: http://yann.lecun.com/exdb/mnist/

## 1.引入工具库

In [1]:
from __future__ import division, print_function, absolute_import
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
session = tf.Session(config=config, ...)

## 2.设定超参数
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

In [2]:
# 训练超参数
learning_rate = 0.001
num_steps = 600
batch_size = 128
display_step = 10

# 神经网络超参数
num_input = 784 # MNIST数据输入(数据形状: 28*28)
num_classes = 10 # MNIST总共类别(0-9总共10个手写数字)
dropout = 0.25 # Dropout随机失活概率


# 占位符
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

# 变量
weights = {
    # 5x5 卷积核, 1 输入, 32 输出
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 卷积核, 32 输入, 64 输出
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # 全连接, 7*7*64 输入, 1024 输出
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 输入, 10 输出(总共1类)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

## 3.准备数据

In [7]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


## 4.构建模型
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

In [4]:
# 定义卷积层运算(卷积层+激活层)
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

# 定义池化层运算
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# 构建卷积神经网络模型
def conv_net(x, weights, biases, dropout):
    # 原本MNIST数据输入是784维度向量
    # 调整成 宽*高*通道数 的标准图片输入格式
    # 总共是4个维度[Batch Size, Height, Width, Channel]，其中第一个维度-1表示暂时不确定一个batch多少张
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # 卷积层
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # 池化层
    conv1 = maxpool2d(conv1, k=2)

    # 卷积层
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # 池化层
    conv2 = maxpool2d(conv2, k=2)

    # 全连接层，需要先把数据展开成一维向量，再接全连接
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # 随机失活层
    fc1 = tf.nn.dropout(fc1, dropout)

    # 最终预估结果
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

## 5.计算损失与优化

In [8]:
# 构建卷积神经网络预估结果
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# 定义损失与优化器
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# 评估模型
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 初始化变量
init = tf.global_variables_initializer()

## 6.在session当中完成计算图计算(损失计算与优化、参数更新迭代)
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

In [6]:
# 在session当中开始训练
with tf.Session() as sess:

    # 初始化
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # 优化
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # 计算准确率
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("第" + str(step) + "步, 一个minibatch上的损失为 " + \
                  "{:.4f}".format(loss) + ", 训练准确率为 " + \
                  "{:.3f}".format(acc))

    print("完成训练！")

    # 计算准确率
    print("测试准确率为 ", \
        sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                      Y: mnist.test.labels[:256],
                                      keep_prob: 1.0}))

第1步, 一个minibatch上的损失为 50938.1016, 训练准确率为 0.094
第10步, 一个minibatch上的损失为 28999.7930, 训练准确率为 0.148
第20步, 一个minibatch上的损失为 12606.2256, 训练准确率为 0.484
第30步, 一个minibatch上的损失为 8245.8047, 训练准确率为 0.547
第40步, 一个minibatch上的损失为 6496.4678, 训练准确率为 0.625
第50步, 一个minibatch上的损失为 3963.8340, 训练准确率为 0.758
第60步, 一个minibatch上的损失为 2181.0652, 训练准确率为 0.781
第70步, 一个minibatch上的损失为 1751.7107, 训练准确率为 0.852
第80步, 一个minibatch上的损失为 3384.8621, 训练准确率为 0.836
第90步, 一个minibatch上的损失为 1731.7202, 训练准确率为 0.844
第100步, 一个minibatch上的损失为 2309.3552, 训练准确率为 0.836
第110步, 一个minibatch上的损失为 1668.9805, 训练准确率为 0.828
第120步, 一个minibatch上的损失为 2588.8555, 训练准确率为 0.805
第130步, 一个minibatch上的损失为 1502.3131, 训练准确率为 0.883
第140步, 一个minibatch上的损失为 887.5152, 训练准确率为 0.930
第150步, 一个minibatch上的损失为 1604.7382, 训练准确率为 0.891
第160步, 一个minibatch上的损失为 1585.2931, 训练准确率为 0.867
第170步, 一个minibatch上的损失为 1846.4788, 训练准确率为 0.844
第180步, 一个minibatch上的损失为 1174.4926, 训练准确率为 0.906
第190步, 一个minibatch上的损失为 841.4255, 训练准确率为 0.914
第200步, 一个minibatch上的损失为 995.9590, 训练准确率为 0.859
第21

### 版权归 © 稀牛学院 所有 保留所有权利
![](./graphs/xiniu_neteasy.png)